In [ ]:
import geopandas as gpd
import os
import psycopg2

In [ ]:
def open_connection():
    conn = \
        psycopg2.connect( \
            host = 'localhost',
            database = 'queroponto',
            user = 'postgres',
            password = 'postgres')
    
    conn.autocommit = True
    cur = conn.cursor()
    
    return conn, cur

In [ ]:
def close_connection(conn, cur):
    cur.close()
    conn.close()

In [ ]:
def get_state_data(state_id):
    data = {
        11: ['RO', 'Rondônia'],
        12: ['AC', 'Acre'],
        13: ['AM', 'Amazonas'],
        14: ['RR', 'Roraima'],
        15: ['PA', 'Pará'],
        16: ['AP', 'Amapá'],
        17: ['TO', 'Tocantins'],
        21: ['MA', 'Maranhão'],
        22: ['PI', 'Piauí'],
        23: ['CE', 'Ceará'],
        24: ['RN', 'Rio Grande do Norte'],
        25: ['PB', 'Paraíba'],
        26: ['PE', 'Pernambuco'],
        27: ['AL', 'Alagoas'],
        28: ['SE', 'Sergipe'],
        29: ['BA', 'Bahia'],
        31: ['MG', 'Minas Gerais'],
        32: ['ES', 'Espírito Santo'],
        33: ['RJ', 'Rio de Janeiro'],
        35: ['SP', 'São Paulo'],
        41: ['PR', 'Paraná'],
        42: ['SC', 'Santa Catarina'],
        43: ['RS', 'Rio Grande do Sul'],
        50: ['MS', 'Mato Grosso do Sul'],
        51: ['MT', 'Mato Grosso'],
        52: ['GO', 'Goiás'],
        53: ['DF', 'Distrito Federal']
    }
    
    return data[state_id][0], data[state_id][1]

In [ ]:
def insert_data(cur, values):
    query = f" \
        INSERT INTO public.estados (cod_ibge, id_regiao, nome, sigla, geom) VALUES ( \
            {values[0]}, \
            {values[1]}, \
            '{values[2]}', \
            '{values[3]}', \
            ST_GeomFromText('{values[4]}', 4326) \
        );"
    
    cur.execute(query)

---

In [ ]:
path = '../../arquivos/ibge/estados'

In [ ]:
folders = os.listdir(path)

In [ ]:
conn, cur = open_connection()

In [ ]:
for folder in folders:
    for file in os.listdir(f"{path}/{folder}"):
        if file.split('.')[-1] == 'shp':
            gdf = gpd.read_file(f"{path}/{folder}/{file}")
            
            for row in gdf.itertuples():
                acronym, name = get_state_data(int(row[2]))
                values = [int(row[2]), int(row[2][:1]), name, acronym, row[5]]
                
                insert_data(cur, values)

In [ ]:
close_connection(conn, cur)